In [ ]:
from main_pl import main as train_model
from main_gmgnn import main as train_model_gngmm
from label_items_with_gmm import main as label_with_gmm
from label_items_with_kmeans import main as label_with_kmeans
from basic_cat_augmentation import main as label_with_categories
from torch import set_float32_matmul_precision
set_float32_matmul_precision('medium')

In [7]:
dataset='diginetica'

In [8]:
base_runs=[
    ## my hparams, previosulsy calculated base runs
    'qm2ur7o3',
    '3abge2uq',
    '4dm99qnd',
    'jxgwsuta',
]

## search for best # clusters

In [9]:
augmented_common_flags=f"--dataset {dataset} --augment-alg gmm --validation --valid-portion 0.1 --step 2 --lr-dc-step 2 --patience 4 --augment-matrix --augment-std 0.01 --augment-normalize --augment-p 0.5 --l2 0.000001 --augment-noise-p 0.5"

In [ ]:
augmented_run_ids=[]
for rid in base_runs:
    for no_gmm in [4, 8, 16, 32, 64, 128]:
        extra_flags=f'--augment-old-run-id {rid} --augment-nogmm {no_gmm} '

        gmm_label_flags=f"--run-id {rid} --no-clusters {no_gmm}"
        label_with_gmm(gmm_label_flags.strip())

        augmented_model_flags=augmented_common_flags+" "+extra_flags
        aug_rid=train_model(
            augmented_model_flags.strip()
        )
        augmented_run_ids.append(aug_rid)

In [ ]:
augmented_run_ids

### kmeans

In [ ]:
augmented_run_ids=[]

augmented_common_flags=f"--dataset {dataset} --augment-alg kmeans --validation --valid-portion 0.1 --step 2 --lr-dc-step 2 --patience 4 --augment-matrix --augment-std 0.01 --augment-normalize --augment-p 0.5 --l2 0.000001 --augment-noise-p 0.5"
for rid in base_runs:
    extra_flags=f'--augment-old-run-id {rid} --augment-nogmm {8} '

    gmm_label_flags=f"--run-id {rid} --no-clusters {8}"
    label_with_kmeans(gmm_label_flags.strip())

    augmented_model_flags=augmented_common_flags+" "+extra_flags
    aug_rid=train_model(
        augmented_model_flags.strip()
    )
    augmented_run_ids.append(aug_rid)



### categories

In [ ]:
augmented_run_ids=[]
augmented_common_flags=f"--dataset {dataset} --augment-alg categories --validation --valid-portion 0.1 --step 2 --lr-dc-step 2 --patience 4 --augment-matrix --augment-std 0.01 --augment-normalize --augment-p 0.5 --l2 0.000001 --augment-noise-p 0.5"
for rid in base_runs:
    extra_flags=f'--augment-old-run-id {rid}'

    gmm_label_flags=f"--run-id {rid}"
    label_with_categories(gmm_label_flags.strip())

    augmented_model_flags=augmented_common_flags+" "+extra_flags
    aug_rid=train_model(
        augmented_model_flags.strip()
    )
    augmented_run_ids.append(aug_rid)

In [ ]:
augmented_run_ids

In [ ]:
augmented_run_ids

## i2i distance

In [ ]:
augmented_run_ids=[]
augmented_common_flags=f"--augment-p 1 --dataset {dataset} --augment-alg raw --validation --valid-portion 0.1 --step 2 --lr-dc-step 2 --patience 4 --augment-matrix --augment-std 0.01 --l2 0.000001 --augment-noise-p 0.5"
for rid in base_runs:
    extra_flags=f'--augment-old-run-id {rid}'

    augmented_model_flags=augmented_common_flags+" "+extra_flags
    aug_rid=train_model(
        augmented_model_flags.strip()
    )
    augmented_run_ids.append(aug_rid)

In [ ]:
augmented_run_ids

In [ ]:
augmented_run_ids=[]
no_gmm=8
augmented_common_flags=f"--dataset {dataset} --augment-alg raw --validation --valid-portion 0.1 --step 2 --lr-dc-step 2 --patience 4 --augment-matrix --augment-std 0.01 --augment-normalize --augment-p 0.5 --l2 0.000001 --augment-noise-p 0.5"

for rid in base_runs:
    extra_flags=f'--augment-old-run-id {rid} --augment-nogmm {no_gmm} '

    gmm_label_flags=f"--run-id {rid} --no-clusters {no_gmm}"
    
    label_with_gmm(gmm_label_flags.strip())

    augmented_model_flags=augmented_common_flags+" "+extra_flags
    aug_rid=train_model_gngmm(
        augmented_model_flags.strip()
    )
    augmented_run_ids.append(aug_rid)
augmented_run_ids

## base GM induced model

In [ ]:
augmented_run_ids=[]
no_gmm=8
base_common_flags=f"--dataset {dataset} --validation --valid-portion 0.1 --step 2 --lr-dc-step 3 --patience 5"

for rid in base_runs:
    extra_flags=f'--augment-old-run-id {rid} --augment-nogmm {no_gmm} '


    augmented_model_flags=base_common_flags+" "+extra_flags
    aug_rid=train_model_gngmm(
        augmented_model_flags.strip()
    )
    augmented_run_ids.append(aug_rid)
augmented_run_ids

# search for hyperparameters

In [ ]:
augment_dist_flag="--augment-clusters" #/'--augment-categories' or '' for i2i

In [ ]:
flag_str=f"--dataset {dataset} --validation --valid-portion 0.1 --lr-scheduler step --step 2 --lr-dc-step 2 --patience 5"
rid=train_model(
    flag_str.strip()
)

In [ ]:
for no_gmm in [2, 8, 32, 128]:
    if augment_dist_flag=="--augment-clusters":
        gmm_label_flags=f"--run-id {rid} --no-clusters {no_gmm}"
        label_with_gmm(gmm_label_flags.strip())

    elif augment_dist_flag=='--augment-categories':
        cat_label_flags=f"--run-id {rid}"
        label_with_categories(cat_label_flags.strip())
        
    augmented_model_flags=f"--dataset {dataset} --validation --valid-portion 0.1 --lr-scheduler step --step 2 --lr-dc-step 2 --patience 4 --augment-matrix --augment-old-run-id {rid} {augment_dist_flag} --augment-std 0.01 --augment-normalize --augment-p 0.5 --l2 0.000001 --augment-nogmm {no_gmm} --augment-noise-p 0.5"
    for i in range(2):
        _=train_model(
            augmented_model_flags.strip()
        )
    

# rerun baseline+augmented K times, compute test metrics

In [2]:
import pickle
from srgnn_datasets import SRGNN_Map_Dataset, SRGNN_sampler
from utils import load_model
import os

from torch.utils.data import DataLoader

import pandas as pd

from pytorch_lightning import Trainer
from math import ceil
from IPython.display import clear_output

In [3]:
dataset='otto-recsys'

In [4]:
k=5

base_run_ids=[]
aug_run_ids=[]

In [ ]:
## noise only
for i in range(k):
    flag_str=f"--dataset {dataset} --validation --valid-portion 0.1 --step 2 --lr-dc-step 2 --patience 5 --augment-noise-p 0.5 --augment-std 0.01"
    rid=train_model(
        flag_str.strip()
    )
    base_run_ids.append(rid)
    clear_output(wait=True)

In [ ]:
#batchSize=256

for i in range(k):
    flag_str=f"--dataset {dataset} --validation --valid-portion 0.1 --step 2 --lr-dc-step 2 --patience 5"
    rid=train_model(
        flag_str.strip()
    )
    base_run_ids.append(rid)

    clustering_label_flags=f"--run-id {rid} --no-clusters {8}"
    label_with_gmm(clustering_label_flags.strip())
    label_with_kmeans(clustering_label_flags.strip())
    if dataset!='otto-recsys':
        label_with_categories(f"--run-id {rid}".strip())

    augmented_common_flags=f"--dataset {dataset} --augment-old-run-id {rid} --augment-nogmm 8 --validation --valid-portion 0.1 --step 2 --lr-dc-step 2 --patience 4 --augment-matrix --augment-std 0.01 --augment-normalize --augment-p 0.5 --l2 0.000001 --augment-noise-p 0.5"

    for alg in ['categories','gmm', 'kmeans', 'raw']:
        if dataset=='otto-recsys' and alg=='categories':
            continue
        alg_flag=f"--augment-alg {alg}"
        
        augmented_model_flags=augmented_common_flags+" "+alg_flag
        aug_rid=train_model(
            augmented_model_flags.strip()
        )
        aug_run_ids.append(aug_rid)
    clear_output(wait=True)

In [ ]:
base_run_ids

In [ ]:
aug_run_ids

In [ ]:
with open('../datasets/' + dataset + '/test.txt', 'rb') as f:
    test_data = pickle.load(f)
normal_test_dataset=SRGNN_Map_Dataset(test_data, shuffle=False)

batchSize=100
normal_test_dataloader=DataLoader(normal_test_dataset, 
                            num_workers=os.cpu_count(),  
                            sampler=SRGNN_sampler(normal_test_dataset, batchSize, shuffle=False, drop_last=False)
                            )

In [ ]:
trainer=Trainer(limit_test_batches=ceil(normal_test_dataset.length/batchSize),
                limit_predict_batches=ceil(normal_test_dataset.length/batchSize))

In [ ]:
base_res_df=[]
for run_id in base_run_ids:
    model,opt=load_model(run_id)

    metrics=trainer.test(model, {'normal':normal_test_dataloader})[0]
    for k,v in opt.__dict__.items():
       metrics[k]=v
    base_res_df.append(metrics)

aug_res_df=[]
for run_id in aug_run_ids:
    model,opt=load_model(run_id)

    metrics=trainer.test(model, {'normal':normal_test_dataloader})[0]
    for k,v in opt.__dict__.items():
       metrics[k]=v
    aug_res_df.append(metrics)   

In [ ]:
res_df=pd.DataFrame(aug_res_df+base_res_df)

In [ ]:
res_df.to_csv(f'./final_results/res_df_{"_".join(res_df.run_id.tolist())}.csv')

In [ ]:
res_df.drop(columns='run_id').groupby('augment_matrix').mean()